# Lesson 5: Training a Reward Model on Mac M3 (Hugging Face)

## 🍎 TTRL on Apple Silicon
So far, we have used "Rule-Based Verifiers" (asking the LLM "Is this correct?").
But in production systems like ChatGPT or DeepSeek, we use **Trained Reward Models**.

A Reward Model is a specialized neural network trained to output a score representing quality. It is faster and more accurate than asking an LLM to chat about itself.

**In this lesson:**
1.  We set up **PyTorch MPS** (Metal Performance Shaders) to train on your Mac M3.
2.  We use Hugging Face `TRL` (Transformer Reinforcement Learning).
3.  We train a **LoRA (Low-Rank Adapter)** to classify "Good Math" vs "Bad Math".

In [ ]:
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from trl import RewardTrainer, RewardConfig
from datasets import load_dataset
from peft import LoraConfig, TaskType, get_peft_model

# 1. Setup MPS (Apple Silicon)
# This checks if your Mac's GPU is available for math acceleration.
device = "mps" if torch.backends.mps.is_available() else "cpu"
print(f"Training on device: {device}")

### 📊 Step 1: The Preference Dataset
Reward models learn from **Comparisons**. They need to know what is "Better".

Structure: `{Prompt, Chosen (Winner), Rejected (Loser)}`

We create a tiny synthetic dataset here to demonstrate the format.

In [ ]:
# We construct a tiny synthetic dataset for demonstration
data = [
    {"prompt": "2+2=", "chosen": "The answer is 4.", "rejected": "The answer is 5."},
    {"prompt": "Is 91 prime?", "chosen": "No, 91 is 7*13.", "rejected": "Yes, 91 is prime."},
    {"prompt": "Capital of France?", "chosen": "Paris", "rejected": "London"}
] * 10 # Repeat 10 times so the trainer has enough batches to run

from datasets import Dataset
dataset = Dataset.from_list(data)
print("Dataset prepared.")

### 🏗️ Step 2: The Model with LoRA
We load `Qwen2.5-0.5B`. This is a very small, fast model perfect for Mac experimentation.

**What is specific here?**
*   `AutoModelForSequenceClassification`: We aren't generating text. We are outputting a single number (Score).
*   `num_labels=1`: The output is a scaler value (Reward).
*   `LoraConfig`: We freeze the gigabytes of main weights, and only train a tiny 10MB adapter. This makes training feasible on a laptop.

In [ ]:
model_name = "Qwen/Qwen2.5-0.5B-Instruct"
print(f"Loading {model_name}...")

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

# Load Model with Classification Head (1 label = Score)
model = AutoModelForSequenceClassification.from_pretrained(
    model_name, 
    num_labels=1, 
    torch_dtype=torch.float16, # Half precision for M3 speed
    device_map=device
)
model.config.pad_token_id = tokenizer.pad_token_id

# Define LoRA Config
peft_config = LoraConfig(
    task_type=TaskType.SEQ_CLS,
    inference_mode=False,
    r=8,
    lora_alpha=32,
    lora_dropout=0.1
)
print("Model and PEFT Config ready.")

### 🚀 Step 3: Training with TRL
We use the `RewardTrainer`. It takes our pairs and uses a **Ranking Loss**.
It adjusts the weights so that `Score(Chosen) > Score(Rejected)`.

In [ ]:
# Use RewardConfig instead of TrainingArguments
training_args = RewardConfig(
    output_dir="./reward_model_output",
    per_device_train_batch_size=2,
    num_train_epochs=1,
    learning_rate=1e-4,
    logging_steps=1,
    use_mps_device=True, # IMPORTANT: Tells HF to use M3 Metal
    bf16=False, 
    fp16=False, # Stabilize MPS
)

trainer = RewardTrainer(
    model=model,
    processing_class=tokenizer,
    args=training_args,
    train_dataset=dataset,
    peft_config=peft_config,
)

print("Starting Training on M3...")
trainer.train()

### 🧪 Step 4: Verification (Inference)
We verify the model learned something.
We pass in the "Wrong" answer and the "Right" answer for the Prime problem.
If the score for "Right" is higher, the training worked.

In [ ]:
def score_answer(prompt, answer):
    inputs = tokenizer(prompt, answer, return_tensors="pt", truncation=True).to(device)
    with torch.no_grad():
        score = model(**inputs).logits[0].cpu().item()
    return score

s1 = score_answer("Is 91 prime?", "No, it is 7*13.")
s2 = score_answer("Is 91 prime?", "Yes it is.")

print(f"Score (Correct): {s1:.4f}")
print(f"Score (Wrong): {s2:.4f}")
if s1 > s2:
    print("SUCCESS: Model prefers the correct answer.")
else:
    print("FAIL: Model still prefers the wrong answer.")